In [1]:
from typing import Optional, Union, Any
from df_engine.core import Context, Actor
from data.appointment_    import appointment
from data.doctor_         import doctor
from data.health_         import health
from data.medicine_       import medicine
from data.diet_           import diet
from data.mental_health_  import mental_health


def healmed_request(ctx: Context, actor: Actor, *args, **kwargs) -> Any:
    request = ctx.last_request
    request = request.lower()
    if re.search(r'\bappointment\b|\bdoctor', request, re.IGNORECASE):
        if re.search(r'\bappointment\b|\bdoctor', request, re.IGNORECASE)[0] == 'appointment':
            return appointment(request)
        else:
            return doctor(request)
    elif re.search(r'\bhealth\b|\bmedicine', request, re.IGNORECASE):
        if re.search(r'\bhealth\b|\bmedicine', request, re.IGNORECASE)[0] == 'health':
            return health(request)
        else:
            return medicine(request)
    elif re.search(r'\bdiet\b|\bmental_health', request, re.IGNORECASE):
        if re.search(r'\bdiet\b|\bmental_health', request, re.IGNORECASE)[0] == 'diet':
            return diet(request)
        else:
            return mental_health(request)
    elif re.search(r'\b|Physicians|Cardiologists|Gastroenterologists|Dentist|ENT specialist|Gynaecologist', request, re.IGNORECASE):
        return  """OK! I have book a appointment of {} Doctor for you Sir""".format(request)
    return "Sure Sir, I thanks for your visit that but your can go other services if needed you can type 'hi'"

In [2]:
import re
import logging
from typing import Optional, Union

from df_engine.core.keywords import TRANSITIONS, RESPONSE
from df_engine.core import Context, Actor
import df_engine.conditions as cnd

logger = logging.getLogger(__name__)

# First of all, to create a dialog agent, we need to create a dialog script.
# Below, `plot` is the dialog script.
# A dialog script is a flow dictionary that can contain multiple plot .
# Plot are needed in order to divide a dialog into sub-dialogs and process them separately.
# For example, the separation can be tied to the topic of the dialog.
# In our example, there is one flow called greeting_flow.

# Inside each flow, we can describe a sub-dialog.
# Here we can also use keyword `LOCAL`, which we have considered in other examples.

# Flow describes a sub-dialog using linked nodes, each node has the keywords `RESPONSE` and `TRANSITIONS`.

# `RESPONSE` - contains the response that the dialog agent will return when transitioning to this node.
# `TRANSITIONS` - describes transitions from the current node to other nodes.
# `TRANSITIONS` are described in pairs:
#      - the node to which the agent will perform the transition
#      - the condition under which to make the transition
plot = {
    "greeting_flow": {
        "start_node": {  # This is an initial node, it doesn't need an `RESPONSE`
            RESPONSE: "",
            TRANSITIONS: {"node1": cnd.regexp(r"\bHi\b|hi|hello|Hello|hey|Hey", re.IGNORECASE)}, 
        },
        "node1": {
            RESPONSE: "Hi I'm Health and medicine Chatbot Assitant, how may help you?",  
            TRANSITIONS: {"node2": cnd.regexp(r"\bappointment|Appointment\b|\bdoctor|Doctor|Physicians|Cardiologists|Gastroenterologists|Dentist|ENT specialist|Gynaecologist", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bhealth|Health\b|\bmedicine|Medicine", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bdiet|Diet\b|\bmental health|Mental health", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bThanks|thanks|Thanking|thanking|Good|good|OK|ok|Ok", re.IGNORECASE)},
        },
        "node2": {
            RESPONSE: healmed_request,
            TRANSITIONS: {"node2": cnd.regexp(r"\bappointment|Appointment\b|\bdoctor|Doctor|Physicians|Cardiologists|Gastroenterologists|Dentist|ENT specialist|Gynaecologist", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bhealth|Health\b|\bmedicine|Medicine", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bdiet|Diet\b|\bmental health|Mental health", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bThanks|thanks|Thanking|thanking|Good|good|OK|ok|Ok", re.IGNORECASE)},
            
        },
        "node3": {
            RESPONSE: healmed_request,
            TRANSITIONS: {"node2": cnd.regexp(r"\bappointment|Appointment\b|\bdoctor|Doctor|Physicians|Cardiologists|Gastroenterologists|Dentist|ENT specialist|Gynaecologist", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bhealth|Health\b|\bmedicine|Medicine", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bdiet|Diet\b|\bmental health|Mental health", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bThanks|thanks|Thanking|thanking|Good|good|OK|ok|Ok", re.IGNORECASE)},
        },
        "node4": {
            RESPONSE: healmed_request,
            TRANSITIONS: {"node2": cnd.regexp(r"\bappointment|Appointment\b|\bdoctor|Doctor|Physicians|Cardiologists|Gastroenterologists|Dentist|ENT specialist|Gynaecologist", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bhealth|Health\b|\bmedicine|Medicine", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bdiet|Diet\b|\bmental_health|Mental_health", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bThanks|thanks|Thanking|thanking|Good|good|OK|ok|Ok", re.IGNORECASE)},
        },
        "node5": {
            RESPONSE: "Welcome Sir/Mam",
            TRANSITIONS: {"node2": cnd.regexp(r"\bappointment|Appointment\b|\bdoctor|Doctor|Physicians|Cardiologists|Gastroenterologists|Dentist|ENT specialist|Gynaecologist", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bhealth|Health\b|\bmedicine|Medicine", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bdiet|Diet\b|\bmental health|Mental health", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bThanks|thanks|Thanking|thanking|Good|good|OK|ok|Ok", re.IGNORECASE),
                         "node6": cnd.regexp(r"\bNo|no|NO|Nothing|nothing", re.IGNORECASE)},
        },
        "node6": {RESPONSE: "bye", TRANSITIONS: {"node1": cnd.exact_match("Hi")}},
        "fallback_node": {  # We get to this node if an error occurred while the agent was running
            RESPONSE: "Sure Sir, I thanks for your visit but your can go other services if needed you can type 'hi'",
            TRANSITIONS: {"node1": cnd.regexp(r"\bHi\b|hi|hello|Hello|hey|Hey", re.IGNORECASE)},
        },
    }
}

# An actor is an object that processes user input replicas and returns responses
# To create the actor, you need to pass the script of the dialogue `plot`
# And pass the initial node `start_label`
# and the node to which the actor will go in case of an error `fallback_label`
# If `fallback_label` is not set, then its value becomes equal to `start_label` by default
actor = Actor(plot, start_label=("greeting_flow", "start_node"), fallback_label=("greeting_flow", "fallback_node"))


# turn_handler - a function is made for the convenience of working with an actor
def turn_handler(
    in_request: str, ctx: Union[Context, str, dict], actor: Actor, true_out_response: Optional[str] = None
):
    # Context.cast - gets an object type of [Context, str, dict] returns an object type of Context
    ctx = Context.cast(ctx)
    # Add in current context a next request of user
    ctx.add_request(in_request)
    # pass the context into actor and it returns updated context with actor response
    ctx = actor(ctx)
    # get last actor response from the context
    out_response = ctx.last_response
    # the next condition branching needs for testing
    if true_out_response is not None and true_out_response != out_response:
        msg = f"in_request={in_request} -> true_out_response != out_response: {true_out_response} != {out_response}"
        raise Exception(msg)
    else:
        logging.info(f"in_request={in_request} ->\n {out_response}")
    return out_response, ctx


# testing
testing_dialog = [
    ("Hi", "Hi I'm Health and medicine Chatbot Assitant, how may help you?"),  # start_node -> node1
    ("I want to book a appointment?","OK! I have book a appointment of Doctor for health checkup for you Sir"),
    ("Physicians", "OK! I have book a appointment of physicians Doctor for you Sir"),  
    ("Thanks", "Welcome Sir/Mam"),  
    ("bye", "Sure Sir, I thanks for your visit but your can go other services if needed you can type 'hi'"),
    ("Hi", "Hi I'm Health and medicine Chatbot Assitant, how may help you?"),
    ("What is health?", "Health refers to the extent of a person’s physical, mental, and social well-being. This definition, taken from the World Health Organization’s treatment of health, emphasizes that health is a complex concept that involves not just the soundness of a person’s body but also the state of a person’s mind and the quality of the social environment in which she or he lives. The quality of the social environment in turn can affect a person’s physical and mental health, underscoring the importance of social factors for these twin aspects of our overall well-being."),  
    ("What is medicine","Medicine is the social institution that seeks both to prevent, diagnose, and treat illness and to promote health as just defined. Dissatisfaction with the medical establishment has been growing.Part of this dissatisfaction stems from soaring health-care costs and what many perceive as insensitive stinginess by the health insurance industry, as the 2009 battle over health-care reform illustrated. Some of the dissatisfaction also reflects a growing view that the social and even spiritual realms of human existence play a key role in health and illness. This view has fueled renewed interestin alternative medicine. We return later to these many issues for the social institution of medicine."),        #
    ("what is mental_health?", "Mental health refers to cognitive, behavioral, and emotional well-being. It is all about how people think, feel, and behave. People sometimes use the term “mental health” to mean the absence of a mental disorder. Mental health can affect daily living, relationships, and physical health.Mental health as a state of well-being in which the individual realizes his or her own abilities, can cope with the normal stresses of life, can work productively and fruitfully, and is able to make a contribution to his or her community."), 
    ("how  can are change my diet?", "Diet is the sum of food consumed by a person or other organism. The word diet often implies the use of specific intake of nutrition for health or weight-management reasons (with the two often being related). Although humans are omnivores, each culture and each person holds some food preferences or some food taboos. This may be due to personal tastes or ethical reasons. Individual dietary choices may be more or less healthy.Diet includes a variety of plant-based and animal-based foods that provide nutrients tothe body. Such nutrients provide the body with energy and keep it running. Nutrients help build and strengthenbones, muscles, and tendons and also regulate body processes (i.e., blood pressure). Water is essential for growth, reproduction and good health. Macronutrients are consumed in relatively large quantities and include proteins, carbohydrates, and fats and fatty acids. Micronutrients – vitamins and minerals – are consumed in relatively smaller quantities, but are essential to body processes."),  
    ("what a doctor", "There is a many doctor like following:\n- Physicians\n- Cardiologists\n- Gastroenterologists\n- Dentist\n- ENT specialist\n- Gynaecologist\nPlease write type of doctor according above way"),    
    ("Ok", "Welcome Sir/Mam"),
    ("bye","Sure Sir, I thanks for your visit but your can go other services if needed you can type 'hi'"),
]


def run_test():
    ctx = {}
    for in_request, true_out_response in testing_dialog:
        _, ctx = turn_handler(in_request, ctx, actor, true_out_response=true_out_response)


# interactive mode
def run_interactive_mode(actor):
    ctx = {}
    while True:
        in_request = input("type your answer: ")
        _, ctx = turn_handler(in_request, ctx, actor)


if __name__ == "__main__":
    logging.basicConfig(
        format="%(asctime)s-%(name)15s:%(lineno)3s:%(funcName)20s():%(levelname)s - %(message)s", level=logging.INFO
    )
    run_test()
    print("<-- Test Successful -->")
#     run_interactive_mode(actor)





2022-01-11 07:22:31,652-           root:104:        turn_handler():INFO - in_request=Hi ->
 Hi I'm Health and medicine Chatbot Assitant, how may help you?
2022-01-11 07:22:31,659-           root:104:        turn_handler():INFO - in_request=I want to book a appointment? ->
 OK! I have book a appointment of Doctor for health checkup for you Sir
2022-01-11 07:22:31,667-           root:104:        turn_handler():INFO - in_request=Physicians ->
 OK! I have book a appointment of physicians Doctor for you Sir
2022-01-11 07:22:31,675-           root:104:        turn_handler():INFO - in_request=Thanks ->
 Welcome Sir/Mam
2022-01-11 07:22:31,689-           root:104:        turn_handler():INFO - in_request=bye ->
 Sure Sir, I thanks for your visit but your can go other services if needed you can type 'hi'
2022-01-11 07:22:31,703-           root:104:        turn_handler():INFO - in_request=Hi ->
 Hi I'm Health and medicine Chatbot Assitant, how may help you?
2022-01-11 07:22:31,726-           root

<-- Test Successful -->


In [ ]:
import re
import logging
from typing import Optional, Union

from df_engine.core.keywords import TRANSITIONS, RESPONSE
from df_engine.core import Context, Actor
import df_engine.conditions as cnd

logger = logging.getLogger(__name__)

# First of all, to create a dialog agent, we need to create a dialog script.
# Below, `plot` is the dialog script.
# A dialog script is a flow dictionary that can contain multiple plot .
# Plot are needed in order to divide a dialog into sub-dialogs and process them separately.
# For example, the separation can be tied to the topic of the dialog.
# In our example, there is one flow called greeting_flow.

# Inside each flow, we can describe a sub-dialog.
# Here we can also use keyword `LOCAL`, which we have considered in other examples.

# Flow describes a sub-dialog using linked nodes, each node has the keywords `RESPONSE` and `TRANSITIONS`.

# `RESPONSE` - contains the response that the dialog agent will return when transitioning to this node.
# `TRANSITIONS` - describes transitions from the current node to other nodes.
# `TRANSITIONS` are described in pairs:
#      - the node to which the agent will perform the transition
#      - the condition under which to make the transition
plot = {
    "greeting_flow": {
        "start_node": {  # This is an initial node, it doesn't need an `RESPONSE`
            RESPONSE: "",
            TRANSITIONS: {"node1": cnd.regexp(r"\bHi\b|hi|hello|Hello|hey|Hey", re.IGNORECASE)}, 
        },
        "node1": {
            RESPONSE: "Hi I'm Health and medicine Chatbot Assitant, how may help you?",  
            TRANSITIONS: {"node2": cnd.regexp(r"\bappointment|Appointment\b|\bdoctor|Doctor|Physicians|Cardiologists|Gastroenterologists|Dentist|ENT specialist|Gynaecologist", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bhealth|Health\b|\bmedicine|Medicine", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bdiet|Diet\b|\bmental health|Mental health", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bThanks|thanks|Thanking|thanking|\bGood|\bgood|OK|ok|Ok", re.IGNORECASE)},
        },
        "node2": {
            RESPONSE: healmed_request,
            TRANSITIONS: {"node2": cnd.regexp(r"\bappointment|Appointment\b|\bdoctor|Doctor|Physicians|Cardiologists|Gastroenterologists|Dentist|ENT specialist|Gynaecologist", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bhealth|Health\b|\bmedicine|Medicine", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bdiet|Diet\b|\bmental health|Mental health", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bThanks|thanks|Thanking|thanking|Good|good|OK|ok|Ok", re.IGNORECASE)},
            
        },
        "node3": {
            RESPONSE: healmed_request,
            TRANSITIONS: {"node2": cnd.regexp(r"\bappointment|Appointment\b|\bdoctor|Doctor|Physicians|Cardiologists|Gastroenterologists|Dentist|ENT specialist|Gynaecologist", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bhealth|Health\b|\bmedicine|Medicine", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bdiet|Diet\b|\bmental health|Mental health", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bThanks|thanks|Thanking|thanking|Good|good|OK|ok|Ok", re.IGNORECASE)},
        },
        "node4": {
            RESPONSE: healmed_request,
            TRANSITIONS: {"node2": cnd.regexp(r"\bappointment|Appointment\b|\bdoctor|Doctor|Physicians|Cardiologists|Gastroenterologists|Dentist|ENT specialist|Gynaecologist", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bhealth|Health\b|\bmedicine|Medicine", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bdiet|Diet\b|\bmental_health|Mental_health", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bThanks|thanks|Thanking|thanking|Good|good|OK|ok|Ok", re.IGNORECASE)},
        },
        "node5": {
            RESPONSE: "Welcome Sir/Mam",
            TRANSITIONS: {"node2": cnd.regexp(r"\bappointment|Appointment\b|\bdoctor|Doctor|Physicians|Cardiologists|Gastroenterologists|Dentist|ENT specialist|Gynaecologist", re.IGNORECASE),
                         "node3": cnd.regexp(r"\bhealth|Health\b|\bmedicine|Medicine", re.IGNORECASE),
                         "node4": cnd.regexp(r"\bdiet|Diet\b|\bmental health|Mental health", re.IGNORECASE),
                         "node5": cnd.regexp(r"\bThanks|thanks|Thanking|thanking|Good|good|OK|ok|Ok", re.IGNORECASE),
                         "node6": cnd.regexp(r"\bNo|no|NO|Nothing|nothing", re.IGNORECASE)},
        },
        "node6": {RESPONSE: "bye", TRANSITIONS: {"node1": cnd.exact_match("Hi")}},
        "fallback_node": {  # We get to this node if an error occurred while the agent was running
            RESPONSE: "Sure Sir, I thanks for your visit but your can go other services if needed you can type 'hi'",
            TRANSITIONS: {"node1": cnd.regexp(r"\bHi\b|hi|hello|Hello|hey|Hey", re.IGNORECASE)},
        },
    }
}

# An actor is an object that processes user input replicas and returns responses
# To create the actor, you need to pass the script of the dialogue `plot`
# And pass the initial node `start_label`
# and the node to which the actor will go in case of an error `fallback_label`
# If `fallback_label` is not set, then its value becomes equal to `start_label` by default
actor = Actor(plot, start_label=("greeting_flow", "start_node"), fallback_label=("greeting_flow", "fallback_node"))


# turn_handler - a function is made for the convenience of working with an actor
def turn_handler(
    in_request: str, ctx: Union[Context, str, dict], actor: Actor, true_out_response: Optional[str] = None
):
    # Context.cast - gets an object type of [Context, str, dict] returns an object type of Context
    ctx = Context.cast(ctx)
    # Add in current context a next request of user
    ctx.add_request(in_request)
    # pass the context into actor and it returns updated context with actor response
    ctx = actor(ctx)
    # get last actor response from the context
    out_response = ctx.last_response
    # the next condition branching needs for testing
    if true_out_response is not None and true_out_response != out_response:
        msg = f"in_request={in_request} -> true_out_response != out_response: {true_out_response} != {out_response}"
        raise Exception(msg)
    else:
        logging.info(f"in_request={in_request} ->\n {out_response}")
    return out_response, ctx


# testing
testing_dialog = [
    ("Hi", "Hi I'm Health and medicine Chatbot Assitant, how may help you?"),  # start_node -> node1
    ("I want to book a appointment, how are you?", "Good. What do you want to talk about?"),  # node1 -> node2
    ("", "Sorry, I can not talk about music now."),  
    ("Ok, goodbye.", "bye"),  
    ("Hi", "Hi, how are you?"), 
    ("What is health", "Ooops"),  # node1 -> node2
    ("stop", "Ooops"),  
    ("What is medicine",),        # node1 -> node2
    ("Hi", "Hi, how are you?"), 
    ("i'm fine, how are you?", "Good. What do you want to talk about?"),  
    ("I want to know the diet", "Sorry, I can not talk about music now."),    
    ("Ok, goodbye.", "bye"),  
]


def run_test():
    ctx = {}
    for in_request, true_out_response in testing_dialog:
        _, ctx = turn_handler(in_request, ctx, actor, true_out_response=true_out_response)


# interactive mode
def run_interactive_mode(actor):
    ctx = {}
    while True:
        in_request = input("type your answer: ")
        _, ctx = turn_handler(in_request, ctx, actor)


if __name__ == "__main__":
    logging.basicConfig(
        format="%(asctime)s-%(name)15s:%(lineno)3s:%(funcName)20s():%(levelname)s - %(message)s", level=logging.INFO
    )
#     run_test()
    run_interactive_mode(actor)





type your answer: hey


2022-01-11 07:23:17,806-           root:104:        turn_handler():INFO - in_request=hey ->
 Hi I'm Health and medicine Chatbot Assitant, how may help you?


type your answer: I want an appointment of doctor


2022-01-11 07:23:43,606-           root:104:        turn_handler():INFO - in_request=I want an appointment of doctor ->
 OK! I have book a appointment of Doctor for health checkup for you Sir


type your answer: which doctor


...

2022-01-11 07:25:19,024-           root:104:        turn_handler():INFO - in_request=how can I change my diet plan ->
 Diet is the sum of food consumed by a person or other organism. The word diet often implies the use of specific intake of nutrition for health or weight-management reasons (with the two often being related). Although humans are omnivores, each culture and each person holds some food preferences or some food taboos. This may be due to personal tastes or ethical reasons. Individual dietary choices may be more or less healthy.Diet includes a variety of plant-based and animal-based foods that provide nutrients tothe body. Such nutrients provide the body with energy and keep it running. Nutrients help build and strengthenbones, muscles, and tendons and also regulate body processes (i.e., blood pressure). Water is essential for growth, reproduction and good health. Macronutrients are consumed in relatively large quantities and include proteins, carbohydrates, and fats and fa

type your answer: I issue of mental_Health


2022-01-11 07:25:35,124-           root:104:        turn_handler():INFO - in_request=I issue of mental_Health ->
 Mental health refers to cognitive, behavioral, and emotional well-being. It is all about how people think, feel, and behave. People sometimes use the term “mental health” to mean the absence of a mental disorder. Mental health can affect daily living, relationships, and physical health.Mental health as a state of well-being in which the individual realizes his or her own abilities, can cope with the normal stresses of life, can work productively and fruitfully, and is able to make a contribution to his or her community.


type your answer: good


2022-01-11 07:26:07,348-           root:104:        turn_handler():INFO - in_request=good ->
 Welcome Sir/Mam


In [ ]:
# request= ("What is Health").lower()
# re.search(r'\bhealth\b|\bmedicine', request, re.IGNORECASE)[0]